In [149]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

edc_protocol = django_apps.get_app_config('edc_protocol')

In [150]:
import datetime
from dateutil.relativedelta import relativedelta

edc_protocol = django_apps.get_app_config('edc_protocol')


today = datetime.datetime.now()
year = today.year
month = today.month

closing_year = edc_protocol.study_close_datetime.date().year
closing_month = edc_protocol.study_close_datetime.date().month

dates = []

while True:
# Get dates between now and closing date of the study
    dates.append(datetime.date(year=year, month=month, day=1))
    today = today + relativedelta(months=+1)
    month = today.month
    year = today.year
    
    if year == closing_year and month == closing_month:
        break

dates

[datetime.date(2022, 7, 1),
 datetime.date(2022, 8, 1),
 datetime.date(2022, 9, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 11, 1),
 datetime.date(2022, 12, 1),
 datetime.date(2023, 1, 1),
 datetime.date(2023, 2, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 4, 1),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 6, 1),
 datetime.date(2023, 7, 1),
 datetime.date(2023, 8, 1),
 datetime.date(2023, 9, 1),
 datetime.date(2023, 10, 1),
 datetime.date(2023, 11, 1),
 datetime.date(2023, 12, 1),
 datetime.date(2024, 1, 1),
 datetime.date(2024, 2, 1),
 datetime.date(2024, 3, 1),
 datetime.date(2024, 4, 1),
 datetime.date(2024, 5, 1),
 datetime.date(2024, 6, 1),
 datetime.date(2024, 7, 1),
 datetime.date(2024, 8, 1),
 datetime.date(2024, 9, 1),
 datetime.date(2024, 10, 1),
 datetime.date(2024, 11, 1),
 datetime.date(2024, 12, 1),
 datetime.date(2025, 1, 1),
 datetime.date(2025, 2, 1),
 datetime.date(2025, 3, 1),
 datetime.date(2025, 4, 1),
 datetime.date(2025, 5, 1)]

In [151]:

# GET cohort A chilren

cohart_a_pids = OnScheduleChildCohortAEnrollment.objects.values_list('subject_identifier', flat=True).distinct()
child_consents = CaregiverChildConsent.objects.filter(subject_identifier__in=list(cohart_a_pids)).order_by('child_dob')
child_consents.count()

144

In [156]:
from dateutil import relativedelta
from calendar import Calendar

# children holder dictionary --> format children['2020-01-01'] = [ChildrenQueryset, ]
children = dict()


for day in dates[1:]:
    '''
    dates start from 2022-07-01 till the close date
    but use dates[1:] because children turning 5 this month will be 5 on the 2022-08-01 
    '''
    children[day.isoformat()] = list()
    
    for consent in child_consents:

        delta = relativedelta.relativedelta(day, consent.child_dob)
        
        if delta.years == 5:    
            children[day.isoformat()].append(consent)




In [153]:
children # check format of the dictionary

{'2022-08-01': [<CaregiverChildConsent: CaregiverChildConsent object (950c210b-2dc7-4f6d-8608-d59e589aa80a)>,
  <CaregiverChildConsent: CaregiverChildConsent object (9bdd93aa-e383-4100-8c9b-acf1b561a4d1)>,
  <CaregiverChildConsent: CaregiverChildConsent object (711ea448-1bc6-425d-bc72-870343774d36)>,
  <CaregiverChildConsent: CaregiverChildConsent object (7ef61f38-7a15-4bfa-bdb0-05ce307e3f42)>,
  <CaregiverChildConsent: CaregiverChildConsent object (be50ebeb-61da-404b-bee3-9b0b10b10626)>,
  <CaregiverChildConsent: CaregiverChildConsent object (58ed5bdb-81cc-40c2-8b5f-0dd512cc5ce2)>,
  <CaregiverChildConsent: CaregiverChildConsent object (3fff9ab2-3d42-498a-868b-1fc90378034a)>,
  <CaregiverChildConsent: CaregiverChildConsent object (862d7800-8ea9-4e6f-90a8-48ed3e86dea2)>,
  <CaregiverChildConsent: CaregiverChildConsent object (fa565b71-6c8e-4e50-828b-f2d2473321dc)>,
  <CaregiverChildConsent: CaregiverChildConsent object (ff2d1b61-372b-43b4-8c96-2c43bfa1261b)>,
  <CaregiverChildConsent: 

In [159]:
from dateutil import relativedelta

for day, consents in children.items():
    '''
    chilren contain a day as a key, but the values is the list of chilren turning
    5 for that specific month
    '''
    original_month = datetime.date.fromisoformat(day) - relativedelta.relativedelta(months=+1)
    today = datetime.datetime.now()
    
    
    for consent in consents:

        if (consent.child_dob.year < today.year) and (consent.child_dob.day > today.day):
            """
            children should be those who are turning 5 from tomorrow and upwards
            """

            # get the birthday date
            next_birth_day = datetime.date(original_month.year, consent.child_dob.month, consent.child_dob.day)

            
            while next_birth_day.weekday() > 4:
                # change move the dob backwards instead of foward,
                # because it will be used by the schedule date
                next_birth_day -= datetime.timedelta(days=1)
            
            # move a schedule date 3 days from the birth day
            schedule_date = next_birth_day - datetime.timedelta(days=3)
            
            while schedule_date.weekday() > 4:
                # if the schedule date is behind move the date even further
                schedule_date -= datetime.timedelta(days=1)
            
            # create and appointment
            ParticipantNote.objects.create(
                subject_identifier = consent.subject_identifier,
                title = 'Follow Up',
                color = 'yellow',
                date = schedule_date,
                description = consent.child_dob
            )
